In [28]:
import psycopg2
import random
from faker import Faker
from datetime import datetime , timedelta

In [29]:
fake = Faker()

In [34]:
# Altere para sua conexão local
conn = psycopg2 .connect(
dbname="postgres" ,
user="postgres" ,
password ="postgres" ,
host="localhost" ,
port="5432"
)
cur = conn.cursor()

In [36]:
# Função para gerar dados aleatórios com limite de comprimento
def truncate (text, max_length ):
    return text[:max_length ]

In [38]:
# Inserir registros na tabela Produtos
produto_ids = []
for _ in range(10000):
    nome = truncate (fake.word().capitalize(), 100)
    descricao = truncate (fake.sentence(), 255)
    preco = round(random.uniform( 10.0, 100.0), 2)
    estoque = random.randint( 1, 100)
    categoria = truncate (fake.word().capitalize(), 255)
    cur.execute(
        "INSERT INTO Produtos (Nome, Descricao, Preco, Estoque, Categoria) VALUES ( %s, %s, %s, %s, %s) RETURNING ID_Produto" ,
    ( nome, descricao , preco, estoque, categoria )
)
    produto_ids .append(cur.fetchone()[ 0])

In [39]:
# Inserir registros na tabela Clientes
cliente_ids = []
for _ in range(10000):
    nome = truncate (fake.name(), 100)
    endereco = truncate (fake.address().replace( '\n', ', '), 255)
    telefone = truncate (fake.phone_number(), 15)
    email = truncate (f"{nome.replace( ' ', '').lower() }_{random.randint( 1, 10000)}@example.com" , 100)
    data_nascimento = fake.date_of_birth( minimum_age =18, maximum_age =90)
    cur.execute(
    "INSERT INTO Clientes (Nome, Endereco, Telefone, Email, Data_Nascimento) VALUES ( %s, %s, %s,%s, %s) RETURNING ID_Cliente" ,
    ( nome, endereco , telefone , email, data_nascimento )
)
    cliente_ids .append(cur.fetchone()[ 0])

In [40]:
# Inserir registros na tabela Pedidos
pedido_ids = []
for _ in range(10000):
    data_pedido = fake.date_between( start_date ='-4y', end_date ='today')
    status = truncate (random.choice([ 'Pendente' , 'Concluído' , 'Cancelado' ]), 20)
    valor_total = round(random.uniform( 50.0, 500.0), 2)
    id_cliente = random.choice( cliente_ids )
    cur.execute(
    "INSERT INTO Pedidos (Data_Pedido, Status, Valor_Total, ID_Cliente) VALUES ( %s, %s, %s, %s)    RETURNING ID_Pedido" ,
    ( data_pedido , status, valor_total , id_cliente )
)
    pedido_ids .append(cur.fetchone()[ 0])

In [41]:
# Inserir registros na tabela Itens_Pedido
for _ in range(10000):
    quantidade = random.randint( 1, 10)
    preco_unitario = round(random.uniform( 10.0, 100.0), 2)
    subtotal = quantidade * preco_unitario
    id_pedido = random.choice( pedido_ids )
    id_produto = random.choice( produto_ids )
    cur.execute(
        "INSERT INTO Itens_Pedido (Quantidade, Preco_Unitario, Subtotal, ID_Pedido, ID_Produto) VALUES (%s, %s, %s, %s, %s)",
        ( quantidade , preco_unitario , subtotal , id_pedido , id_produto )
)

In [42]:
# Inserir registros na tabela Pagamentos
for _ in range(10000):
    data_pagamento = fake.date_between( start_date ='-4y', end_date ='today')
    valor_pago = round(random.uniform( 50.0, 500.0), 2)
    metodo_pagamento = truncate (random.choice([ 'Cartão de Crédito' , 'Boleto' , 'Pix']), 50)
    id_pedido = random.choice( pedido_ids )
    cur.execute(
        "INSERT INTO Pagamentos (Data_Pagamento, Valor_Pago, Metodo_Pagamento, ID_Pedido) VALUES ( %s,%s, %s, %s)",
        ( data_pagamento , valor_pago , metodo_pagamento , id_pedido )
)

In [43]:
conn.commit()
cur.close()
conn.close()

* Respondendo pergutas de negócio

In [48]:
# Altere para sua conexão local
conn = psycopg2 .connect(
dbname="postgres" ,
user="postgres" ,
password ="postgres" ,
host="localhost" ,
port="5432"
)
cur = conn.cursor()

In [44]:
import sqlite3
import pandas as pd

In [51]:
query = "SELECT * FROM Clientes;"
clientes = pd.read_sql(query,conn)
clientes

C:\Users\User\AppData\Local\Temp\ipykernel_22804\461974725.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  clientes = pd.read_sql(query,conn)


,id_cliente,nome,endereco,telefone,email,data_nascimento
0,1,Kevin Pierce,"6821 Adams Point Suite 822, Port Luisborough, ...",860.522.9243,kevinpierce_8162@example.com,1971-09-01
1,2,Laura Rodriguez,"155 Debra Curve, Laurenborough, SC 88096",9526469700,laurarodriguez_9179@example.com,1999-01-27
2,3,Edward Anderson,"USCGC Robinson, FPO AA 11635",(797)833-2500,edwardanderson_6676@example.com,1984-10-23
3,4,Patrick Mcintyre,"9061 Jessica Route, West Christina, AK 72621",356.883.3930x49,patrickmcintyre_4071@example.com,1972-06-02
4,5,Jessica Estrada,"940 Franklin Union, Jenniferhaven, RI 68186",(674)855-2211,jessicaestrada_1930@example.com,1965-10-26
...,...,...,...,...,...,...
9995,9996,Michael Watson,"1247 Jackson Plain, West Jerry, LA 56459",568.776.5056,michaelwatson_6395@example.com,1958-06-18
9996,9997,Joshua Crosby,"798 Michael Cape Apt. 411, Garciafort, ND 64865",(363)516-1211x4,joshuacrosby_3640@example.com,1978-05-06
9997,9998,Sabrina Stone PhD,"9674 Kevin Fall, East Rachael, UT 78489",9712080498,sabrinastonephd_1278@example.com,1949-06-25
9998,9999,Jeremy Davis,"6352 Shelby Rest, Lake Marie, PW 23803",001-657-421-296,jeremydavis_9852@example.com,1995-10-06


1 - Selecione todos produtos de uma categoria.

In [57]:
query = """ 
    SELECT *
    FROM Produtos
    WHERE categoria = 'Training'; """

produtos_de_uma_categoria = pd.read_sql(query,conn)
produtos_de_uma_categoria

C:\Users\User\AppData\Local\Temp\ipykernel_22804\4200526751.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  produtos_de_uma_categoria = pd.read_sql(query,conn)


,id_produto,nome,descricao,preco,estoque,categoria
0,1761,Sea,Drive step red.,33.95,21,Training
1,3770,Position,Occur not attorney must executive hospital able.,17.02,44,Training
2,5735,Prepare,Once others nor believe there thing statement.,19.70,93,Training
3,8048,Customer,Election shoulder their require economic party...,27.07,32,Training
4,8607,Election,Win indeed body themselves.,62.18,7,Training
5,8770,Own,Day people positive window once.,35.72,67,Training
6,9996,Film,Strong radio national.,18.57,1,Training


2 - Encontre o nome e o endereço dos clientes que fizeram um pedido em 2024.

In [68]:
query = """
    SELECT
        Clientes.Nome,
        Clientes.Endereco
    FROM
        Clientes
    JOIN
        Pedidos ON Clientes.ID_Cliente = Pedidos.ID_Cliente
    WHERE
        EXTRACT(YEAR FROM Pedidos.Data_Pedido) = 2024;
"""

clientes_que_fizeram_pedidos_2024 = pd.read_sql(query, conn)
clientes_que_fizeram_pedidos_2024

C:\Users\User\AppData\Local\Temp\ipykernel_22804\3553417421.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  clientes_que_fizeram_pedidos_2024 = pd.read_sql(query, conn)


,nome,endereco
0,Patrick Leach,"7501 Rose Flat, Lisamouth, GU 98797"
1,Catherine Cox,"311 Alan Hollow, East Amystad, GU 90090"
2,Richard Ellis,"40141 Gutierrez Island, West Kayleemouth, SD 7..."
3,Dr. Shannon Jordan,"3556 Jones Estate, Rileychester, AR 07902"
4,Jessica Castro,"9563 Dean Rapid Suite 432, West Carlosview, MN..."
...,...,...
1488,Gerald Berg,"249 Patrick Land Apt. 653, Rodrigueztown, NY 7..."
1489,Debbie Alvarado,"9415 Michael Manors Suite 747, Rebeccaside, RI..."
1490,Nicholas Martin,"USNS Carter, FPO AA 88557"
1491,Rhonda Sandoval,"6091 Benitez Field Suite 348, Arthurstad, NJ 3..."


3 - Liste os produtos cujo preço está entre 50 e 100.

In [69]:
query = """ 
    SELECT *
    FROM Produtos
    WHERE Preco BETWEEN 50 AND 100;
"""

preco_produtos_entre_50_e_100 = pd.read_sql(query, conn)
preco_produtos_entre_50_e_100

C:\Users\User\AppData\Local\Temp\ipykernel_22804\3809450673.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  preco_produtos_entre_50_e_100 = pd.read_sql(query, conn)


,id_produto,nome,descricao,preco,estoque,categoria
0,1,Identify,Chance professor follow kitchen.,57.60,8,Current
1,3,Suggest,Reveal might be actually grow explain.,63.12,61,Development
2,4,Positive,Someone during mother will can.,92.18,40,Church
3,5,Born,Media left role pull report.,62.32,62,Radio
4,6,Identify,Policy military while she.,62.95,30,Environment
...,...,...,...,...,...,...
5600,9991,Election,Finish push involve rule place home send.,50.07,42,Realize
5601,9993,Student,Rule involve same real Mrs edge type.,62.28,88,Interview
5602,9994,Time,Floor nothing face especially need window.,64.51,83,Old
5603,9995,Unit,Use material course range.,66.32,21,Score


4 - Conte o número total de pedidos feitos no mês de julho de 2024.

In [70]:
query = """ 
    SELECT
        COUNT(*) AS total_pedidos_julho_2024
    FROM
        Pedidos
    WHERE
        EXTRACT(YEAR FROM Data_Pedido) = 2024
        AND EXTRACT(MONTH FROM Data_Pedido) = 7 
"""

pedidos_julho_2024 = pd.read_sql(query, conn)
pedidos_julho_2024

C:\Users\User\AppData\Local\Temp\ipykernel_22804\2115375178.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pedidos_julho_2024 = pd.read_sql(query, conn)


,total_pedidos_julho_2024
0,195


5 - Encontre o total pago por cada pedido.

In [71]:
query = """
    SELECT
        ID_Pedido,
        SUM(Valor_Pago) AS total_pago
    FROM
        Pagamentos
    GROUP BY
        ID_Pedido;
"""

total_pago_por_cada_pedido = pd.read_sql(query, conn)
total_pago_por_cada_pedido

C:\Users\User\AppData\Local\Temp\ipykernel_22804\48862078.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  total_pago_por_cada_pedido = pd.read_sql(query, conn)


,id_pedido,total_pago
0,1489,402.15
1,6114,717.76
2,273,319.66
3,6977,326.20
4,3936,109.43
...,...,...
6359,9180,74.40
6360,3858,944.78
6361,790,394.87
6362,3984,255.08


6 - Liste o nome dos clientes e o valor total dos seus pedidos.

In [72]:
query = """
    SELECT
        Clientes.Nome AS Nome_Cliente,
        SUM(Pedidos.Valor_Total) AS Valor_Total_Pedidos
    FROM
        Clientes
    JOIN
        Pedidos ON Clientes.ID_Cliente = Pedidos.ID_Cliente
    GROUP BY
        Clientes.Nome;
"""

clientes_valor_total_pedidos = pd.read_sql(query, conn)
clientes_valor_total_pedidos

C:\Users\User\AppData\Local\Temp\ipykernel_22804\2454125728.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  clientes_valor_total_pedidos = pd.read_sql(query, conn)


,nome_cliente,valor_total_pedidos
0,Brandon Jackson,352.96
1,Todd Wilson,304.72
2,Heather Hayes,750.26
3,Michelle Howard,244.29
4,Candace Perkins,192.00
...,...,...
6059,Amy Delgado,182.51
6060,James Bauer,366.96
6061,Jessica Castro,153.13
6062,Russell Walker,445.12


7 - Encontre os produtos com estoque menor que 10.

In [74]:
query = """
    SELECT
        ID_Produto,
        Nome,
        Estoque
    FROM
        Produtos
    WHERE
        Estoque < 10;
"""

estoque_menor_de_10 = pd.read_sql(query, conn)
estoque_menor_de_10

C:\Users\User\AppData\Local\Temp\ipykernel_22804\1001253567.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  estoque_menor_de_10 = pd.read_sql(query, conn)


,id_produto,nome,estoque
0,1,Identify,8
1,18,Social,5
2,49,Decade,9
3,58,End,6
4,59,Yet,8
...,...,...,...
885,9957,Next,7
886,9962,Citizen,7
887,9974,Father,8
888,9990,Line,6


8 - Liste os pedidos com seu status e a quantidade de itens em cada pedido.

In [76]:
query = """
    SELECT
        Pedidos.ID_Pedido,
        Pedidos.Status,
        COUNT(Itens_Pedido.ID_Item) AS Quantidade_Itens
    FROM
        Pedidos
    LEFT JOIN
        Itens_Pedido ON Pedidos.ID_Pedido = Itens_Pedido.ID_Pedido
    GROUP BY
        Pedidos.ID_Pedido, Pedidos.Status;
"""

Pedidos_Status_quantidade_porPedido = pd.read_sql(query, conn)
Pedidos_Status_quantidade_porPedido

C:\Users\User\AppData\Local\Temp\ipykernel_22804\4095546480.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Pedidos_Status_quantidade_porPedido = pd.read_sql(query, conn)


,id_pedido,status,quantidade_itens
0,6114,Cancelado,0
1,4790,Pendente,1
2,273,Pendente,0
3,3936,Pendente,1
4,5761,Cancelado,0
...,...,...,...
9995,4827,Concluído,2
9996,7227,Pendente,1
9997,790,Cancelado,2
9998,5642,Cancelado,0


9 - Encontre os produtos que não foram vendidos em nenhum pedido.

In [80]:
query = """
    SELECT
        Produtos.ID_Produto,
        Produtos.Nome,
        Produtos.Descricao
    FROM
        Produtos
    LEFT JOIN
        Itens_Pedido ON Produtos.ID_Produto = Itens_Pedido.ID_Produto
    WHERE
        Itens_Pedido.ID_Produto IS NULL;
"""

produtos_nao_vendidos = pd.read_sql(query, conn)
produtos_nao_vendidos

C:\Users\User\AppData\Local\Temp\ipykernel_22804\1712142.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  produtos_nao_vendidos = pd.read_sql(query, conn)


,id_produto,nome,descricao
0,1,Identify,Chance professor follow kitchen.
1,2,Him,Send could question ever.
2,6,Identify,Policy military while she.
3,9,Fear,Eat mean add beat.
4,11,Gas,Where enjoy administration team inside black w...
...,...,...,...
3655,9990,Line,What stock strategy south quite success central.
3656,9992,Sister,Mother miss Mr leave.
3657,9993,Student,Rule involve same real Mrs edge type.
3658,9996,Film,Strong radio national.


10 - Liste o nome dos 10 maiores clientes e a quantidade de pedidos feitos por cada um. (por quantidade de pedidos)

In [82]:
query = """
    SELECT
        Clientes.Nome AS Nome_Cliente,
        COUNT(Pedidos.ID_Pedido) AS Quantidade_Pedidos
    FROM
        Clientes
    LEFT JOIN
        Pedidos ON Clientes.ID_Cliente = Pedidos.ID_Cliente
    GROUP BY
        Clientes.Nome
    ORDER BY
        Quantidade_Pedidos DESC
    LIMIT 10;
"""

maiores_10_clientes_por_quantidade = pd.read_sql(query, conn)
maiores_10_clientes_por_quantidade

C:\Users\User\AppData\Local\Temp\ipykernel_22804\1417927887.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  maiores_10_clientes_por_quantidade = pd.read_sql(query, conn)


,nome_cliente,quantidade_pedidos
0,Michael Martin,9
1,Lisa Johnson,8
2,Jennifer Jones,8
3,Melissa Bowers,8
4,Michael Smith,8
5,Michael Ford,7
6,Anne Perry,7
7,Susan Mendoza,7
8,Matthew Allen,7
9,John Young,7


11 - Encontre o produto mais caro e o mais barato.

In [84]:
query_mais_caro = """
    SELECT
        ID_Produto,
        Nome,
        Preco
    FROM
        Produtos
    WHERE
        Preco = (SELECT MAX(Preco) FROM Produtos);
"""

query_mais_barato = """
    SELECT
        ID_Produto,
        Nome,
        Preco
    FROM
        Produtos
    WHERE
        Preco = (SELECT MIN(Preco) FROM Produtos);
"""

produto_mais_caro = pd.read_sql(query_mais_caro, conn)
produto_mais_barato = pd.read_sql(query_mais_barato, conn)

print("Produto Mais Caro:")
print(produto_mais_caro)
print("\nProduto Mais Barato:")
print(produto_mais_barato)


Produto Mais Caro:
   id_produto   nome  preco
0        1450  Share  100.0

Produto Mais Barato:
   id_produto     nome  preco
0        9166  Because  10.01


C:\Users\User\AppData\Local\Temp\ipykernel_22804\2791428446.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  produto_mais_caro = pd.read_sql(query_mais_caro, conn)
C:\Users\User\AppData\Local\Temp\ipykernel_22804\2791428446.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  produto_mais_barato = pd.read_sql(query_mais_barato, conn)


12 - Calcule a idade média dos clientes.

13 - Liste os clientes que possuem o mesmo sobrenome.

14 - Encontre os pedidos feitos por clientes cujo nome começa com a letra 'A'.

In [98]:
query = """
    SELECT
        Pedidos.ID_Pedido,
        Pedidos.Data_Pedido,
        Pedidos.Status,
        Pedidos.Valor_Total,
        Clientes.Nome AS Nome_Cliente
    FROM
        Pedidos
    JOIN
        Clientes ON Pedidos.ID_Cliente = Clientes.ID_Cliente
    WHERE
        Clientes.Nome LIKE 'A%';
"""

pedidos_clientes_nome_que_comecam_com_a_letra_A = pd.read_sql(query, conn)
pedidos_clientes_nome_que_comecam_com_a_letra_A

C:\Users\User\AppData\Local\Temp\ipykernel_22804\1486033477.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pedidos_clientes_nome_que_comecam_com_a_letra_A = pd.read_sql(query, conn)


,id_pedido,data_pedido,status,valor_total,nome_cliente
0,10,2022-10-12,Pendente,486.22,Anthony Chaney
1,14,2021-07-12,Concluído,298.64,Alexander Spencer
2,54,2024-07-22,Pendente,248.97,Ashley Sharp
3,88,2023-12-09,Concluído,214.74,Alexander Mann
4,91,2021-03-16,Cancelado,471.02,Ashley Terry
...,...,...,...,...,...
970,9961,2022-09-05,Concluído,122.07,Angela Trevino
971,9966,2020-09-03,Cancelado,122.37,Andrea Williams
972,9969,2022-11-09,Concluído,449.02,Arthur Marquez
973,9975,2021-09-22,Pendente,446.37,Allison Johnson


15 - Liste os produtos, sua categoria e uma indicação se estão em estoque ou esgotados (usando CASE).

In [99]:
query = """
    SELECT
    Nome AS Nome_Produto,
    Categoria,
    CASE
        WHEN Estoque > 0 THEN 'Em estoque'
        ELSE 'Esgotado'
    END AS Status_Estoque
FROM
    Produtos;
"""

lista_produtos_EmEstoque_SemEstoque = pd.read_sql(query, conn)
lista_produtos_EmEstoque_SemEstoque

C:\Users\User\AppData\Local\Temp\ipykernel_22804\1307541078.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  lista_produtos_EmEstoque_SemEstoque = pd.read_sql(query, conn)


,nome_produto,categoria,status_estoque
0,Identify,Current,Em estoque
1,Him,Green,Em estoque
2,Suggest,Development,Em estoque
3,Positive,Church,Em estoque
4,Born,Radio,Em estoque
...,...,...,...
9995,Film,Training,Em estoque
9996,Good,Society,Em estoque
9997,Daughter,Sense,Em estoque
9998,National,High,Em estoque
